In [14]:
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("MNIST_data", one_hot=True)
%matplotlib inline

Extracting MNIST_data\train-images-idx3-ubyte.gz
Extracting MNIST_data\train-labels-idx1-ubyte.gz
Extracting MNIST_data\t10k-images-idx3-ubyte.gz
Extracting MNIST_data\t10k-labels-idx1-ubyte.gz


In [51]:
total_epoch = 100
batch_size = 100
learning_rate = 0.0002
n_hidden = 256 #hidden layer 갯수
n_input = 28*28 #image size
n_noise = 128 # noise size

In [52]:
X = tf.placeholder(tf.float32,[None, n_input]) # 원래 data (batch_size,784)
Z = tf.placeholder(tf.float32,[None,n_noise]) # noise (batch_size,n_noise)

In [53]:
#generat 변수들
#은닉층 출력을 위한 가중치,편향
G_W1 = tf.Variable(tf.random_normal([n_noise,n_hidden],stddev = 0.01))
G_b1 = tf.Variable(tf.zeros([n_hidden]))
#출력층 출력을 위한 가중치,편향
G_W2 = tf.Variable(tf.random_normal([n_hidden,n_input],stddev = 0.01))
G_b2 = tf.Variable(tf.zeros([n_input]))

In [54]:
#difcriminator 변수들
#은닉층 출력을 위한 가중치,편향
D_W1 = tf.Variable(tf.random_normal([n_input,n_hidden],stddev = 0.01))
D_b1 = tf.Variable(tf.zeros([n_hidden]))
#출력층 출력을 위한 가중치,편향
D_W2 = tf.Variable(tf.random_normal([n_hidden,1],stddev = 0.01))
D_b2 = tf.Variable(tf.zeros([1]))

In [55]:
def generator(noise_z):
    hidden = tf.nn.relu(tf.matmul(noise_z,G_W1)+G_b1)
    output = tf.nn.sigmoid(tf.matmul(hidden,G_W2)+G_b2)
    return output

In [56]:
def discriminator(inputs):
    hidden = tf.nn.relu(tf.matmul(inputs,D_W1)+D_b1)
    output = tf.nn.sigmoid(tf.matmul(hidden,D_W2)+D_b2) # output을 0과 1 사이의 값으로 만들어줌
    return output

In [57]:
#무작위 noise를 batch_size만큼 만들어줌
def get_noise(batch_size,n_noise):
    return np.random.normal(size=(batch_size,n_noise))

In [58]:
G = generator(Z)# noise로 부터 생성된 이미지
D_gene = discriminator(G) #가짜 이미지 판별값
D_real = discriminator(X) #진짜 이미지 판별값

In [59]:
# discriminator loss
loss_D = tf.reduce_mean(tf.log(D_real) + tf.log(1-D_gene))

In [60]:
# generator loss
loss_G = tf.reduce_mean(tf.log(D_gene))

In [61]:
#각각 신경망 변수들 구분해줌
D_var_list = [D_W1, D_b1, D_W2, D_b2]
G_var_list = [G_W1, G_b1, G_W2, G_b2]

In [62]:
# training 방법 설정
train_D = tf.train.AdamOptimizer(learning_rate).minimize(-loss_D,var_list=D_var_list)#maximise 하기위해 loss에 음수 붙여줌
train_G = tf.train.AdamOptimizer(learning_rate).minimize(-loss_G,var_list=G_var_list)

In [63]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())

total_batch = int(mnist.train.num_examples / batch_size)
loss_val_D,loss_val_G=0,0 #loss D와 loss_G의 결과값을 받을 변수

In [49]:
for epoch in range(total_epoch): # 100번
    for i in range(total_batch): #550번
        batch_xs, batch_ys = mnist.train.next_batch(batch_size)# batch_xs= (100,784) , batch_ys=(100,10)
        noise = get_noise(batch_size,n_noise)
        
        _,loss_val_D = sess.run([train_D,loss_D],feed_dict={X:batch_xs, Z:noise})
        _,loss_val_G = sess.run([train_G,loss_G],feed_dict={Z:noise})
        
        print('Epoch:', ' %04d' %epoch,'D loss:{:.4}'.format(loss_val_D),'G loss:{:.4}'.format(loss_val_G))

        # 생성된 이미지 저장
        if epoch == 0 or (epoch +1) % 10 == 0:
            sample_size = 10
            noise = get_noise(sample_size, n_noise) # size 128 noise 를 10개씩 만듬
            samples = sess.run(G, feed_dict={Z: noise})
            
            fig, ax = plt.subplots(1, sample_size, figsize = {sample_size, 1})
            
            for i in range(sample_size):
                ax[i].set_axis_off()
                ax[i].imshow(np.reshape(samples[i],(28,28)))
                
            plt.savefig('samples/{}.png'.format(str(epoch).zfill(3)),bbox_inches='tight')
            plt.close(fig)
print('최적화 완료!')

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ...

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ...

KeyboardInterrupt: 

원하는 숫자 만들기

In [1]:
import tensorflow as tf
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np

from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("MNIST_data", one_hot=True)

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data\train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data\train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting MNIST_data\t10k-images-idx3-ubyte.gz
Extracting MNIST_data\t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


In [2]:
total_epoch = 100
batch_size = 100
n_hidden = 256
n_input = 28*28
n_noise = 128
n_class = 10

X = tf.placeholder(tf.float32, [None,n_input])
Y = tf.placeholder(tf.float32, [None,n_class]) # 노이즈와 실제 이미지에 힌트를 넣어줌
Z = tf.placeholder(tf.float32, [None,n_noise])

In [3]:
def generator(noise, labels):
    with tf.variable_scope('generator'):
        inputs = tf.concat([noise, labels],1)
        hidden = tf.layers.dense(inputs,n_hidden,activation=tf.nn.relu)
        output = tf.layers.dense(hidden, n_input,activation=tf.nn.sigmoid)
    return output

In [4]:
def discriminator(inputs, labels,reuse=None):
    with tf.variable_scope('discriminator') as scope:
        if reuse:
            scope.reuse_variables()
            
        inputs = tf.concat([inputs,labels],1)
        hidden = tf.layers.dense(inputs,n_hidden,activation=tf.nn.relu)
        output = tf.layers.dense(hidden,1,activation=None)
    return output

In [5]:
def get_noise(batch_size,n_noise):
    return np.random.uniform(-1.,1.,size=[batch_size,n_noise])

In [6]:
G = generator(Z,Y)
D_real = discriminator(X,Y)
D_gene = discriminator(G,Y,True) # 진짜와 가짜 이미지를 판별할 때 이전의 변수를 재사용해줌

In [7]:
loss_D_real = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=D_real, labels=tf.ones_like(D_real)))#1과 비교

loss_D_gene = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=D_gene, labels=tf.zeros_like(D_gene)))#0 과 비교

loss_D = loss_D_real + loss_D_gene


In [9]:
loss_G = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=D_gene, labels=tf.ones_like(D_gene)))

In [11]:
vars_D = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES,scope='discriminator')
vars_G = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES,scope='generator')

train_D = tf.train.AdamOptimizer().minimize(loss_D,var_list=vars_D)
train_G = tf.train.AdamOptimizer().minimize(loss_G,var_list=vars_G)

In [14]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())

total_batch = int(mnist.train.num_examples / batch_size)
loss_val_D, loss_val_G = 0,0

for epoch in range(total_epoch):
    for i in range(total_batch):
        batch_xs, batch_ys = mnist.train.next_batch(batch_size)
        noise = get_noise(batch_size, n_noise)
        
        _,loss_val_D = sess.run([train_D,loss_D],feed_dict={X:batch_xs, Y:batch_ys, Z:noise})
        _,loss_val_G = sess.run([train_G,loss_G],feed_dict={Y:batch_ys,Z:noise})
        
    print('Epoch:', '%04d' %epoch,'D loss" {:.4}'.format(loss_val_D),'G loss: {:.4}'.format(loss_val_G))
    
    if epoch == 0 or (epoch+1)%10 == 0:
        sample_size =10
        noise = get_noise(sample_size, n_noise)
        samples = sess.run(G,feed_dict = {Y:mnist.test.labels[:sample_size],Z:noise})
        
        fig, ax = plt.subplots(2, sample_size,figsize=(sample_size,2))
        
        for i in range(sample_size):
            ax[0][i].set_axis_off()
            ax[1][i].set_axis_off()
            
            ax[0][i].imshow(np.reshape(mnist.test.images[i],(28,28)))
            ax[1][i].imshow(np.reshape(samples[i],(28,28)))
    plt.savefig('samples2/{}.png'.format(str(epoch).zfill(3)),bbox_inches='tight')
    plt.close(fig)
print('최적화 완료!')

Epoch: 0000 D loss" 0.0166 G loss: 6.875
Epoch: 0001 D loss" 0.008349 G loss: 7.199
Epoch: 0002 D loss" 0.01165 G loss: 6.283
Epoch: 0003 D loss" 0.01988 G loss: 6.127
Epoch: 0004 D loss" 0.07886 G loss: 7.058
Epoch: 0005 D loss" 0.02775 G loss: 5.795
Epoch: 0006 D loss" 0.0212 G loss: 6.779
Epoch: 0007 D loss" 0.0878 G loss: 8.069
Epoch: 0008 D loss" 0.1357 G loss: 6.212
Epoch: 0009 D loss" 0.1769 G loss: 5.534
Epoch: 0010 D loss" 0.4884 G loss: 4.982
Epoch: 0011 D loss" 0.1434 G loss: 4.61
Epoch: 0012 D loss" 0.2014 G loss: 5.101
Epoch: 0013 D loss" 0.3064 G loss: 3.444
Epoch: 0014 D loss" 0.4863 G loss: 4.075
Epoch: 0015 D loss" 0.4143 G loss: 4.18
Epoch: 0016 D loss" 0.3798 G loss: 3.443
Epoch: 0017 D loss" 0.5941 G loss: 4.146
Epoch: 0018 D loss" 0.593 G loss: 3.416
Epoch: 0019 D loss" 0.6293 G loss: 3.106
Epoch: 0020 D loss" 0.7931 G loss: 2.592
Epoch: 0021 D loss" 0.5068 G loss: 3.37
Epoch: 0022 D loss" 0.625 G loss: 2.639
Epoch: 0023 D loss" 0.5187 G loss: 3.077
Epoch: 0024 D l